In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings(action="ignore")
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
stock = pd.read_csv("/kaggle/input/yahoo-stock-price/CSV.csv")
stock.head(10)

In [ ]:
stock.shape

In [ ]:
import missingno as msno
msno.matrix(stock)

* There are no null values

# AAPL (Apple) Stock Price for Stock Price Prediction Demonstration

In [ ]:
from pandas_datareader.data import DataReader
# For time stamps
from datetime import datetime
# getting data from yahoo
df = DataReader('AAPL', data_source='yahoo', start='2010-01-03', end=datetime.now())
df.head(10)

In [ ]:
# plotting Close price history
plt.figure(figsize=(14,8))
plt.plot(df["Close"])
plt.title("Close Price History")
plt.xlabel("Date")
plt.ylabel("Close Price USD($)")
plt.show()

In [ ]:
data = df.filter(["Close"])
dataset = data.values

# Getting rows to train model
training_len = int(np.ceil(len(dataset)*0.95))
training_len

# Apply MinMaxScaler
* LSTMs are highly sensitive to the data which is passed through it. So, scaling data to range-(0,1).

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
scaled_data = sc.fit_transform(dataset)
scaled_data

# Creating Training Data

In [ ]:
train_data = scaled_data[:int(training_len),:]

X_train, y_train = [], []

for i in range(60,len(train_data)):
    X_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])
    if i<=61:
        print(X_train)
        print(y_train)

X_train, y_train = np.array(X_train), np.array(y_train)
#changing shape of data from 2D to 3D
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))
print("X_train shape:{}".format(X_train.shape))

# Model Training...🚴

In [ ]:
model = Sequential()
model.add(LSTM(units=128, return_sequences=True, input_shape= (X_train.shape[1], 1)))
model.add(LSTM(units=32, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(X_train, y_train, batch_size=1, epochs=1, verbose=True)

# Creating Test Data

In [ ]:
test_data = scaled_data[training_len - 60: , :]
# Create the data sets x_test and y_test
x_test = []
y_test = dataset[training_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])
    
# Convert the data to a numpy array
x_test = np.array(x_test)

# Reshape the data
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1 ))

# Get the models predicted price values 
predictions = model.predict(x_test)
predictions = sc.inverse_transform(predictions)

In [ ]:
# Get the root mean squared error (RMSE)
rmse = np.sqrt(np.mean(((predictions - y_test) ** 2)))
print("Root Mean Squared Error: {}".format(rmse))

# Visualize Data

In [ ]:
training = data[:training_len]
valid = data[training_len:]
valid["Predictions"] = predictions
valid

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(training,label="Training")
plt.plot(valid.iloc[:,:],label=["Valid","Predicted"])
plt.title("Close Price History")
plt.xlabel("Year")
plt.ylabel("Close Price in USD($)")
plt.legend()
plt.show()

# Predict 30 Days Future Price

In [ ]:
x_input=test_data[len(test_data)-100:].reshape(1,-1)
x_input.shape

In [ ]:
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

In [ ]:
from numpy import array

lst_output=[]
n_steps=100
i=0
while(i<30):
    
    if(len(temp_input)>100):
        x_input=np.array(temp_input[1:])
        x_input=x_input.reshape(1,-1)
        x_input = x_input.reshape((1, n_steps, 1))
        #print(x_input)
        yhat = model.predict(x_input, verbose=0)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat[0].tolist())
        temp_input=temp_input[1:]
        #print(temp_input)
        lst_output.extend(yhat.tolist())
        i=i+1
    else:
        x_input = x_input.reshape((1, n_steps,1))
        yhat = model.predict(x_input, verbose=0)
        #print(yhat[0])
        temp_input.extend(yhat[0].tolist())
        #print(len(temp_input))
        lst_output.extend(yhat.tolist())
        i=i+1
    

In [ ]:
print(sc.inverse_transform(lst_output))

In [ ]:
day_new=np.arange(1,101)
day_pred=np.arange(101,131)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(day_new,data[len(data)-100:])
plt.plot(day_pred,sc.inverse_transform(lst_output))

<div class="alert alert-box alert-warning">
UPVOTE the notebook if you find it useful and your suggestions are welcome in the comments section. 

See ya!
</div>